3

In [1]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Missions à remplir
missions = [
    "Faire un post pour présenter @product",
    "Faire une story à propos de @product",
    "Prendre une photo de vous avec @product"
]

# Réponse à vérifier
response = "J'ai acheté @product hier et je suis très satisfait de mon achat. J'ai publié une photo avec @product sur Instagram et j'ai partagé une story à propos de mon expérience avec @product."

# Expression régulière pour extraire les tags et les mentions
pattern = r"(#\w+)|(@\w+)"

# Extraire les tags et les mentions de la réponse
response_tags_mentions = set(re.findall(pattern, response))

# Extraire le texte de la réponse
response_text = re.sub(pattern, "", response).strip()

# Vectoriser le texte des missions et de la réponse
vectorizer = TfidfVectorizer(stop_words='english')
mission_vectors = vectorizer.fit_transform(missions)
response_vector = vectorizer.transform([response_text])

# Entraîner un modèle de classification logistique pour prédire la mission appropriée
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
clf.fit(mission_vectors, np.arange(len(missions)))
prediction = clf.predict(response_vector)[0]

# Afficher la mission prédite
print(f"La réponse correspond à la mission : {missions[prediction]}")

La réponse correspond à la mission : Prendre une photo de vous avec @product


In [5]:
import re

# Missions à remplir
missions = [
    "Faire un post pour présenter @product",
    "Faire une story à propos de @product",
    "Prendre une photo de vous avec @product"
]

# Réponse à vérifier
response = "J'ai acheté @product hier et je suis très satisfait de mon achat. J'ai publié une photo avec @product sur Instagram et j'ai partagé une story à propos de mon expérience avec @product."

# Expression régulière pour extraire les tags et les mentions
pattern = r"(#\w+)|(@\w+)"

# Extraire les tags et les mentions de la réponse
response_tags_mentions = set(re.findall(pattern, response))

# Extraire le texte de la réponse
response_text = re.sub(pattern, "", response).strip()

# Initialiser le score de validation à 0
validation_score = 0

# Vérifier chaque mission
for mission in missions:
    # Extraire les tags et les mentions de la mission
    mission_tags_mentions = set(re.findall(pattern, mission))
    # Vérifier si tous les tags et mentions de la mission sont présents dans la réponse
    if mission_tags_mentions.issubset(response_tags_mentions):
        # Ajouter un point si tous les tags et mentions sont présents
        validation_score += 1
    # Vérifier si le texte de la mission est présent dans la réponse
    if mission_text := re.sub(pattern, "", mission).strip():
        print(mission_text)
        print(response_text)
        if mission_text in response_text:
            print("in")
            # Ajouter un point si le texte de la mission est présent
            validation_score += 1

# Afficher le score de validation
print(f"Le score de validation pour la réponse est : {validation_score}/{len(missions)*2}")


Faire un post pour présenter
J'ai acheté  hier et je suis très satisfait de mon achat. J'ai publié une photo avec  sur Instagram et j'ai partagé une story à propos de mon expérience avec .
Faire une story à propos de
J'ai acheté  hier et je suis très satisfait de mon achat. J'ai publié une photo avec  sur Instagram et j'ai partagé une story à propos de mon expérience avec .
Prendre une photo de vous avec
J'ai acheté  hier et je suis très satisfait de mon achat. J'ai publié une photo avec  sur Instagram et j'ai partagé une story à propos de mon expérience avec .
Le score de validation pour la réponse est : 3/6


#  result

In [8]:
import json
import requests
from transformers import AutoTokenizer, AutoModel
import re
import numpy as np
import torch
import tweepy
from decouple import config

def get_twitter_poste(id_poste,access_token,access_token_secret):
        consumer_key = config('TWITTER_CONSUMER_KEY')
        consumer_secret = config('TWITTER_CONSUMER_SECRET')
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        tweet = api.get_status(id_poste)
        tweet = tweet._json
        return tweet['text']
def get_tiktok_poste(id_poste,refrech_token):
        getUrl = f"https://open-api.tiktok.com/oauth/refresh_token?client_key={config('TIKTOK_KEY')}&grant_type=refresh_token&refresh_token={refrech_token}"
        resMedia = requests.get(getUrl)
        accessToken = json.loads(resMedia.text)['data']['access_token']
        api_endpoint = "https://open.tiktokapis.com/v2/video/query/?fields=duration,title,video_description"
        # Set your headers
        headers = {
            "Authorization":"Bearer " + accessToken,
            "Content-Type": "application/json"
        }
        # Set your payload (data)
        payload = {
            "filters": {
                "video_ids": [
                    id_poste,
                    
                ]
            }
        }
        response = requests.post(api_endpoint, headers=headers, data=payload)
        v = response.json()['data']['videos'][0]
        return v["video_description"]+" "+v["titre"]+" "+v["duration"]
        
missions = [
"Write a positive sentence about SaTT",
"Include #SaTT #crypto #influencer #Post2Earn #SocialFi ",
"Include “Earn crypto with your social networks” in your post"
]
# Réponse à vérifier
response = "J'ai acheté @product #SaTT #crypto  hier et je suis très satisfait de mon achat."
def text_confirm(mission,response):
    # Chargement du modèle de traitement du langage naturel BERT
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = AutoModel.from_pretrained("bert-base-uncased")

    def encode_text(text):
        tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**tokens)
            encoded_text = outputs[0][:, 0, :].numpy()
        return encoded_text

    # Expression régulière pour extraire les tags et les mentions
    pattern = r"([@#]\w+)"
    lien_pattern= re.compile(
            r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
            r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')
    # Extraire les tags et les mentions de la réponse
    response_tags_mentions = set(re.findall(pattern, response))
    response_lien = set(re.findall(lien_pattern, response))
    # Extraire le texte de la réponse
    response_text = re.sub(pattern, "", response).strip()
    response_text = re.sub(lien_pattern, "", response_text).strip()
    # Initialiser le score de validation à 0
    validation_score = 0
    total=0
    # Vérifier chaque mission
    for mission in missions:
        # Extraire les tags et les mentions de la mission
        mission_tags_mentions = set(re.findall(pattern, mission))
        total+=len(mission_tags_mentions)
    
        mission_lien = set(re.findall(lien_pattern, mission))
        total+=len(mission_lien)
    
        # Vérifier si tous les tags et mentions de la mission sont présents dans la réponse
        validation_score +=len(mission_tags_mentions.intersection(response_tags_mentions))
        validation_score +=len(mission_lien.intersection(response_lien))
        mission_text = re.sub(pattern,"", mission).strip()
        mission_text = re.sub(lien_pattern,"", mission).strip()
        mission_vector = encode_text(mission_text)
        response_vector = encode_text(response_text)
        # Calculate cosine similarity
        similarity = np.dot(mission_vector,  response_vector.T) / (np.linalg.norm(mission_vector) * np.linalg.norm( response_vector.T))
        total+=1
        if similarity > 0.7:
            # Ajouter un point si la similarité est suffisante
            validation_score += 1
    # Afficher le score de validation
    return validation_score *5 / total

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Write a positive sentence about SaTT 0
Include #SaTT #crypto #influencer #Post2Earn #SocialFi  6
Include “Earn crypto with your social networks” in your post 7
Faire une story à propos de @product 9
Le score de validation pour la réponse est : 2.5/5


In [34]:
from transformers import AutoTokenizer, AutoModel
import re
import numpy as np
import torch

# twitter 
# Missions à remplir
missions = [
"Write a positive sentence about SaTT",
"Include #SaTT #crypto #influencer #Post2Earn #SocialFi",
"Include “Earn crypto with your social networks” in your post"
]

missions = [
    "Faire un post pour présenter @product",
    "Faire une story à propos de @product",
    "Prendre une photo de vous avec @product"
]
# Réponse à vérifier
response = "J'ai acheté @product #SaTT #crypto  hier et je suis très satisfait de mon achat. J'ai publié une photo avec @product sur Instagram et j'ai partagé une story à propos de mon expérience avec @product."

# Chargement du modèle de traitement du langage naturel BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def encode_text(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
        encoded_text = outputs[0][:, 0, :].numpy()
    return encoded_text

# Expression régulière pour extraire les tags et les mentions
pattern = r"([@#]\w+)"

# Extraire les tags et les mentions de la réponse
response_tags_mentions = set(re.findall(pattern, response))

# Extraire le texte de la réponse
response_text = re.sub(pattern, "", response).strip()

# Initialiser le score de validation à 0
validation_score = 0
total=0
# Vérifier chaque mission
for mission in missions:
    # Extraire les tags et les mentions de la mission
    mission_tags_mentions = set(re.findall(pattern, mission))
    total+=len(mission_tags_mentions)
    # Vérifier si tous les tags et mentions de la mission sont présents dans la réponse
    validation_score +=len(mission_tags_mentions.intersection(response_tags_mentions))
    # if mission_tags_mentions and response_tags_mentions.issubset(mission_tags_mentions):
    #     print("in")
    #     # Ajouter un point si tous les tags et mentions sont présents
    #     validation_score += 1
    mission_text = re.sub(pattern, "", mission).strip()
    mission_vector = encode_text(mission_text)
    response_vector = encode_text(response_text)
    # similarity = np.dot(mission_vector, response_vector.T)
    # print(similarity)
    # Calculate cosine similarity
    similarity = np.dot(mission_vector,  response_vector.T) / (np.linalg.norm(mission_vector) * np.linalg.norm( response_vector.T))
    total+=1
    print(similarity)
    if similarity > 0.7:
        # Ajouter un point si la similarité est suffisante
        validation_score += 1
# Afficher le score de validation
print(f"Le score de validation pour la réponse est : {validation_score}/{total}")

ModuleNotFoundError: No module named 'gensim'

In [8]:
import os
import openai
import tweepy
from decouple import config

oracle=0
id_poste=0
def get_poste(oracle,id_poste):
    desc=""
    if oracle == 0:
        consumer_key = config('TWITTER_CONSUMER_KEY')
        consumer_secret = config('TWITTER_CONSUMER_SECRET')
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        # auth.set_access_token(access_key,access_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
    elif oracle == 1:
        oracle=0
    return desc   


    
def text_confirm():
  mission=[]
  poste=get_poste(oracle,id_poste)
  openai.api_key = config('OpenAI')
  start_sequence = "\nA:"
  restart_sequence = "\n\nQ: "
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Q:donnes un score sur 5  pour la confirmiter de ce poste a ces mission :\nposte: {poste} \n mission:{mission}\nA:",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
  )
  response = list(response)
  return response["choices"][0]["text"]

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " 4/5"
    }
  ],
  "created": 1684158984,
  "id": "cmpl-7GSyWKZ6gfiG4ysiZcFBwuiADKEgj",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 129,
    "total_tokens": 132
  }
}
